# 卷积神经网络

## 图像识别的特性

平移不变性（translation invariance）：不管检测对象出现在图像中的哪个位置，神经网络的前面几层应该对相同的图像区域具有相似的反应，即为“平移不变性”。

局部性（locality）：神经网络的前面几层应该只探索输入图像中的局部区域，而不过度在意图像中相隔较远区域的关系，这就是“局部性”原则。最终，可以聚合这些局部特征，以在整个图像级别进行预测。

## 从全连接层到卷积神经网络

一般的来说 设计全连接层 我们需要思维的张量(从(k, l) -> (i, j))的一一映射

形式的

$$H_{i, j} = b_{i, j} + \sum_{k} \sum_{l} w_{i, j, k, l} a_{k, l}$$
$$H_{i, j} = b_{i, j} + \sum_{a} \sum_{b} v_{i, j, a, b} a_{a+i, b+j}$$
从数学的角度利用`平移不变性`和`局部性` 两个特点

### 平移不变性

由平移不变性 我们可以知道 $v_{i, j, a, b} = v_{a, b}$ 及 平移的过程中, 卷积的`kernel`核函数 是不会变的

### 局部性

局部性告诉我们 并不是需要枚举所有的范围,我们只需要枚举一个局部,及$a, b \in [-\Delta, \Delta]$

## 形式化的
我们得到
$$H_{i, j} = b_{i, j} + \sum_{a=-\Delta}^{\Delta} \sum_{b=-\Delta}^{\Delta} v_{a, b} a_{a+i, b+j}$$

所以 我们只需要训练获得$V_{a, b}, a, b \in [-\Delta, \Delta]$

## 更进一步的
图像还有一个`C`作为参数,及通道数 (如图像一般由`R,G,B`三个通道组成)所以我们需要转变形式,为了实现多输入多输出我们得到

$$H_{i, j, d} = b_{i, j} + \sum_{a=-\Delta}^{\Delta} \sum_{b=-\Delta}^{\Delta} \sum_{c} v_{a, b, c, d} a_{a+i, b+j, c}$$

## 通过训练得到核函数

`e.g.` 检测垂直`0, 1`边缘

In [2]:
import torch
from d2l import torch as d2l
from torch import nn

In [3]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w] * K).sum()
    return Y

X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [195]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super.__init__()
        self.weight = nn.Parameter(kernel_size)
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

X = torch.ones((6, 8))
X[:, 2:6] = 0
Kernel = torch.tensor([[1, -1]])
Y = corr2d(X, Kernel)
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)
print(X, '\n', Y, '\n', Y.shape)

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]]) 
 tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]) 
 torch.Size([6, 7])


In [222]:
lr = 0.03
num_epoch = 10
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
for i in range(num_epoch):
    y_hat = conv2d(X)
    loss = (Y - y_hat) ** 2
    conv2d.zero_grad()
    loss.sum().backward()
    with torch.no_grad():
        conv2d.weight.data -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {loss.sum():.3f}')
print(conv2d.weight.data.reshape(1, 2))

epoch 2, loss 0.000
epoch 4, loss 0.000
epoch 6, loss 0.000
epoch 8, loss 0.000
epoch 10, loss 0.000
tensor([[ 1.0000, -1.0000]])


## 步长 stride 填充 padding

In [235]:
X = torch.rand(size=(8, 8))
def comp_conv2d(conv2d, X):
    # 这里的（1，1）表示批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道
    return Y.reshape(Y.shape[2:])

conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1) # api = padding 上下左右填充1
X = torch.rand(size=(8, 8))
print(comp_conv2d(conv2d, X).shape)

conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
print(comp_conv2d(conv2d, X).shape)

conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
print(comp_conv2d(conv2d, X).shape)

conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
print(comp_conv2d(conv2d, X).shape)

torch.Size([8, 8])
torch.Size([8, 8])
torch.Size([4, 4])
torch.Size([2, 2])


## 多输入多输出通道

In [246]:
def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)
print(X, '\n', K)

def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)
K = torch.stack((K, K + 1, K + 2), 0)
print(K.shape)
print(corr2d_multi_in_out(X, K))

tensor([[[0., 1., 2.],
         [3., 4., 5.],
         [6., 7., 8.]],

        [[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]]]) 
 tensor([[[0., 1.],
         [2., 3.]],

        [[1., 2.],
         [3., 4.]]])
torch.Size([3, 2, 2, 2])
tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])


## 池化层

https://blog.csdn.net/qq_40507857/article/details/119854085


一句话概括`cat`和`stack`的区别的话就是:`cat`之后维度是不变的,`stack`之后维度会增加1

[2, 3]按照dim=0 `cat`得到的是[4, 3], `stack`得到的是`[2, 2, 3]`

In [249]:
import torch
from torch import nn
from d2l import torch as d2l

def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
print(pool2d(X, (2, 2)))

print(pool2d(X, (2, 2), 'avg'))

tensor([[4., 5.],
        [7., 8.]])
tensor([[2., 3.],
        [5., 6.]])


In [254]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
pool2d = nn.MaxPool2d((2, 3), stride=(2, 3), padding=(0, 1))
print(pool2d(X))
X = torch.cat((X, X + 1), 1)
X.shape

tensor([[[[ 5.,  7.],
          [13., 15.]]]])


torch.Size([1, 2, 4, 4])

在 PyTorch 中，张量的形状由 torch.Size([...]) 表示，每个维度具有特定的意义，特别是在处理卷积神经网络（CNN）时。对于给定的张量形状 torch.Size([1, 2, 4, 4])，其含义如下：

`第一维度 (1)`: 这通常表示批大小`(batch size)`，即一次处理的样本数量。在这个例子中，批大小为1，表示一次只处理一个样本。

`第二维度 (2)`: 在大多数CNN应用中，第二维度代表通道数`(channels)`。这可以是输入通道的数量，如果这是输入数据的张量的话。在这个例子中，它表示有2个输入通道。

`第三和第四维度 (4, 4)`: 这两个维度通常表示每个通道的空间维度，即图像的高度和宽度。在这里，每个通道的图像大小是4x4像素。